#### Challenge 56:  RC4 Single-Byte Biases

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
%matplotlib inline

import numpy as np
import numpy.matlib 
import base64
import matplotlib.pyplot as plt
from Crypto.Cipher import ARC4
from Crypto import Random

<div class="alert alert-block alert-info">   

RC4 is popular stream cipher notable for its usage in protocols like TLS, WPA, RDP, &c.

It's also susceptible to significant single-byte biases, especially early in the keystream. What does this mean?

Simply: for a given position in the keystream, certain bytes are more (or less) likely to pop up than others. Given enough encryptions of a given plaintext, an attacker can use these biases to recover the entire plaintext.

Now, search online for ["On the Security of RC4 in TLS and WPA"](http://www.isg.rhul.ac.uk/tls/). This site is your one-stop shop for RC4 information.

Click through to ["RC4 biases"](http://www.isg.rhul.ac.uk/tls/biases.pdf) on the right.

These are graphs of each single-byte bias (one per page). Notice in particular the monster spikes on `z16`, `z32`, `z48`, etc. (Note: these are _one-indexed_, so `z16 = keystream[15]`.)

How useful are these biases?

Click through to the research paper and scroll down to the simulation results. (Incidentally, the whole paper is a good read if you have some spare time.) We start out with clear spikes at `2^26` iterations, but our chances for recovering each of the first `256` bytes approaches `1` as we get up towards `2^32`.

There are two ways to take advantage of these biases. The first method is really simple:

1. Gain exhaustive knowledge of the keystream biases.
2. Encrypt the unknown plaintext 2^30+ times under different keys.
3. Compare the ciphertext biases against the keystream biases.

Doing this requires deep knowledge of the biases for each byte of the keystream. But it turns out we can do pretty well with just a few useful biases - if we have some control over the plaintext.

How? By using knowledge of a single bias as a peephole into the plaintext.

</div>    

<div class="alert alert-block alert-info">  

Decode this secret:

`QkUgU1VSRSBUTyBEUklOSyBZT1VSIE9WQUxUSU5F`

And call it a cookie. No peeking!

</div>    

<div class="alert alert-block alert-info">  
    
Now use it to build this encryption oracle:

`RC4(your-request || cookie, random-key)`

Use a fresh 128-bit key on every invocation.

</div>  

In [2]:
# ****** UNKNOWN PARAMETERS
secret_cookie_b64 = 'QkUgU1VSRSBUTyBEUklOSyBZT1VSIE9WQUxUSU5F'
secret_cookie = base64.b64decode(secret_cookie_b64)
# ****** UNKNOWN PARAMETERS

def Challenge56_Oracle(request):
    
    key = Random.get_random_bytes(16)
    cipher = ARC4.new(key)
    return(cipher.encrypt(request + secret_cookie))
    

  
<div class="alert alert-block alert-info">  
    
Picture this scenario: you want to steal a user's secure cookie. You can spawn arbitrary requests (from a malicious plugin or somesuch) and monitor network traffic. (Ok, this is unrealistic - the cookie wouldn't be right at the beginning of the request like that - this is just an example!)

You can control the position of the cookie by requesting `"/"`, `"/A"`, `"/AA"`, and so on.

Build bias maps for a couple chosen indices (`z16` and `z32` are good) and decrypt the cookie.

</div>

In [3]:
Num_Iterations = 2**26

# See what length is if we provide a null "request"
secret_length = len(Challenge56_Oracle(b''))

map15 = np.zeros((secret_length, 256))
map31 = np.zeros((secret_length, 256))
map47 = np.zeros((secret_length, 256))

There are a lot of interesting things we could do here, but I'll just start with the naive approach:

1.  Make a large # of requests with the secret cookie in a particular position.  2**24 per position seems to be enough to recover most bytes of plaintext.
2.  Log the value of the ciphertext produced at each byte position (15, 31, 47, ...) known to have a large key byte bias towards a significant value
3.  Recover the plaintext by finding what PT byte would result in a peak bias that matches the known RC4 bias (remember, CT is just PT xored with KY)

In [4]:
Num_Iterations = 2**24
for pad_length in range(15, 31):
    
    print(f'{pad_length}, ', end='')
    request = b'\x00'*pad_length
    
    b_15 = 15 - pad_length  # Bias peak is 240
    b_31 = 31 - pad_length  # Bias peak is 224
    b_47 = 47 - pad_length  # Bias peak is 208
    
    for jj in range(Num_Iterations):
        
        ciphertext = Challenge56_Oracle(request)
        if pad_length < 16:
            map15[b_15][ciphertext[15]] += 1
        if pad_length < 32 and len(ciphertext) >= 32:
            map31[b_31][ciphertext[31]] += 1
        if len(ciphertext) >= 48:
            map47[b_47][ciphertext[47]] += 1
        
    pad_length += 1

15, 

KeyboardInterrupt: 

In [128]:
m15_maxes = [np.argmax(map15[ii]) for ii in range(len(map15))]
m31_maxes = [np.argmax(map31[ii]) for ii in range(len(map31))]
m47_maxes = [np.argmax(map47[ii]) for ii in range(len(map47))]

print(m15_maxes)
print(m31_maxes)
print(m47_maxes)

m15_pt = bytes([c ^ 240 for c in m15_maxes])
m31_pt = bytes([c ^ 224 for c in m31_maxes])
m47_pt = bytes([c ^ 208 for c in m47_maxes])

print(m15_pt)
print(m31_pt)
print(m47_pt)

# Combine the data collected at the high bias byte positions...15, 31, 47, etc...
combined = np.zeros((secret_length, 256))
for ii in range(secret_length):
    for jj in range(256):
        combined[ii][jj^240] += int(map15[ii][jj])
        combined[ii][jj^224] += int(map31[ii][jj]) 
        combined[ii][jj^208] += int(map47[ii][jj])
    
msg = ''
for ii in range(secret_length):
    msg += chr(combined[ii].argmax())

print('*****************************')
print('Decrypted cookie:')
print(msg)

[178, 181, 208, 163, 165, 162, 181, 208, 164, 191, 208, 180, 162, 185, 190, 187, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 192, 185, 175, 117, 178, 192, 175, 182, 161, 172, 180, 169, 174, 165]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
b'BE SURE TO DRINK\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0\xf0'
b'\xe0\xe0\xe0\xe0\xe0\xe0\xe0\xe0\xe0\xe0\xe0\xe0\xe0\xe0\xe0\xe0 YO\x95R OVALTINE'
b'\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0\xd0'
*****************************
Decrypted cookie:
BE SURE TO DRINK YOR OVALTINE


In [3]:
import pickle
f = open('rc4dist.pickle', 'rb')
key_prob_mat = pickle.load(f)
f.close()

In [5]:
Num_Iterations = 2**17

# Initialize the plaintext probability matrix.
secret_length = len(Challenge56_Oracle(b''))
pt_prior_prob = np.matlib.repmat(np.array(1/128), secret_length, 128)

max_pad_length = 47

filename = './pt_prob_mat2.mmp'
pt_prob_mat = np.memmap(filename, mode="w+", dtype=np.float, shape=(secret_length, 128, Num_Iterations*max_pad_length))
#pt_prob_mat = np.zeros((secret_length, 128, Num_Iterations*max_pad_length))
pt_prob_mat[:,:,0] = pt_prior_prob

max_length = max_pad_length + secret_length
index_mat = np.tile(range(128), (max_length, 1))

count = 0
for pad_length in range(0, 21):
    
    print(f'{pad_length}, ', end='')
    request = b'\x00'*pad_length
        
    for jj in range(1, Num_Iterations):
        
        ciphertext = np.array(list(Challenge56_Oracle(request)))
        ciphertext_len = len(ciphertext)        
        ciphertext_mat = np.tile(ciphertext, (128, 1)).T
        hyp_ky_mat = np.bitwise_xor(ciphertext_mat, index_mat[:ciphertext_len,:])
        likelihood_mat = np.zeros((secret_length, 128))
        
        for kk in range(ciphertext_len):
            likelihood_mat[kk-pad_length,:] = key_prob_mat[kk, [hyp_ky_mat[kk,:]]]
                                                
        bayes_numerators = likelihood_mat * pt_prob_mat[:,:,count]
        bayes_denominators = np.tile(np.sum(bayes_numerators, axis=1).T, (128, 1)).T
        count += 1
        pt_prob_mat[:,:,count] = bayes_numerators / bayes_denominators
    
    pt_prob_mat.flush()
        

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 

In [57]:
Num_Iterations = 2**16
for pad_length in range(0, 21):
    
    print(f'{pad_length}, ', end='')
    request = b'\x00'*pad_length
        
    for jj in range(1, Num_Iterations):
        
        ciphertext = np.array(list(Challenge56_Oracle(request)))
        ciphertext_len = len(ciphertext)        
        ciphertext_mat = np.tile(ciphertext, (128, 1)).T
        hyp_ky_mat = np.bitwise_xor(ciphertext_mat, index_mat[:ciphertext_len,:])
        likelihood_mat = np.zeros((secret_length, 128))
        
        for kk in range(ciphertext_len):
            likelihood_mat[kk-pad_length,:] = key_prob_mat[kk, [hyp_ky_mat[kk,:]]]
                                                
        bayes_numerators = likelihood_mat * pt_prob_mat[:,:,count]
        bayes_denominators = np.tile(np.sum(bayes_numerators, axis=1).T, (128, 1)).T
        count += 1
        pt_prob_mat[:,:,count] = bayes_numerators / bayes_denominators
    
    pt_prob_mat.flush()

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 

In [5]:
Num_Iterations = 2**17

# Initialize the plaintext probability matrix.
secret_length = len(Challenge56_Oracle(b''))
pt_prior_prob = np.matlib.repmat(np.array(1/128), secret_length, 128)

max_pad_length = 47

filename = './pt_prob_mat2.mmp'
pt_prob_mat = np.memmap(filename, mode="w+", dtype=np.float, shape=(secret_length, 128, Num_Iterations*max_pad_length))
pt_prob_mat[:,:,0] = pt_prior_prob

max_length = max_pad_length + secret_length
index_mat = np.tile(range(128), (max_length, 1))

count = 0
for pad_length in range(0, 21):
    
    print(f'{pad_length}, ', end='')
    request = b'\x00'*pad_length
        
    for jj in range(1, Num_Iterations):
        
        ciphertext = np.array(list(Challenge56_Oracle(request)))
        ciphertext_len = len(ciphertext)        
        ciphertext_mat = np.tile(ciphertext, (128, 1)).T
        hyp_ky_mat = np.bitwise_xor(ciphertext_mat, index_mat[:ciphertext_len,:])
        likelihood_mat = np.zeros((secret_length, 128))
        
        for kk in range(ciphertext_len):
            likelihood_mat[kk-pad_length,:] = key_prob_mat[kk, [hyp_ky_mat[kk,:]]]
                                                
        bayes_numerators = likelihood_mat * pt_prob_mat[:,:,count]
        bayes_denominators = np.tile(np.sum(bayes_numerators, axis=1).T, (128, 1)).T
        count += 1
        pt_prob_mat[:,:,count] = bayes_numerators / bayes_denominators
    
    pt_prob_mat.flush()
        

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [28]:
del pt_prob_mat